# AM216 Final-Project: Extend Mini-Project: Predicting drug target interaction using 3D information and predicting ligands that can bind to COVID-19 targets.

Components of X:
*   **Protein**: residues chains (vector length 1000 or other lengths we choose to truncate)
*   **Protein 3D**: contact map of residue distances (vector length around ~20000 after flattening the triangular matrix; need to standardize)
*   **Ligand**: ECFP signature (1024 or 2048)

y: affinity score

Proposed models:
1.   **Mini project**: concatenate protein residue chain and ligant signature to get X
2.   **Just contact map**: concatenate flattened contact map and ligand signature to get X
3.   **Early fusion**: concatenate protein residue chain, flattened contact map and ligand signature to get X
4.   **Late fusion**: train and predict using Model 1 and Model 2 separately, consensus model at the end
5.   **CNN**: Instead of flattening contact map, use convolutaional NN to convolve the map and derive flattened layer
6.   **Pocket**: Instead of using the contact map of the whole protein, get the residues of the just the pocket and get the contact map










## 1. Install Packages

*   Rdkit
*   Deepchem
* Lifelines (calculate CI scores faster)
* Biopython (BioPython's **Bio.PDB** module includes code to load PDB files and calculate the alpha carbon distance between pair of their residues)





In [31]:
%tensorflow_version 1.x

In [32]:
%%capture cap --no-stderr 
#%%capture is used to suppress the output of this particular cell 

import sys
import time
import os.path
if not os.path.exists('Anaconda3-2019.10-Linux-x86_64.sh'):
  t_start = time.time()
  !wget -c https://repo.anaconda.com/archive/Anaconda3-2019.10-Linux-x86_64.sh
  !chmod +x Anaconda3-2019.10-Linux-x86_64.sh
  !bash ./Anaconda3-2019.10-Linux-x86_64.sh -b -f -p /usr/local
  print("Done: Anaconda3", file=sys.stderr)
  t_end = time.time(); print("----Elasped: %f seconds" % (t_end - t_start), file=sys.stderr); t_start = time.time();


  !conda install -y -c deepchem -c rdkit -c conda-forge -c omnia deepchem-gpu=2.3.0
  import deepchem as dc
  print("Done: deepchem & rdkit", file=sys.stderr)
  # !conda create -c conda-forge -n my-rdkit-env rdkit;
  # !activate my-rdkit-env; #(OR TRY: !activate my-rdkit-env)
  # !conda install -c conda-forge rdkit;
  # print("Done: rdkit", file=sys.stderr)
  t_end = time.time(); print("----Elasped: %f seconds" % (t_end - t_start), file=sys.stderr); t_start = time.time();

  !pip install lifelines;
  !pip install biopython;
  !pip install mmtf-python;
  print("Done: pip install lifelines & biopython & mmtf-python", file=sys.stderr)
  t_end = time.time(); print("----Elasped: %f seconds" % (t_end - t_start), file=sys.stderr);

Done: Anaconda3
----Elasped: 116.495961 seconds


ModuleNotFoundError: ignored

In [33]:
%%capture
import sys
sys.path.append('/usr/local/lib/python3.7/site-packages/')
import warnings
warnings.filterwarnings("ignore")


import numpy as np
import os.path
import urllib.request
import pandas as pd
import Bio.PDB
import tensorflow as tf


from rdkit.Chem import AllChem as Chem
from rdkit.Chem import MolFromSmiles
from Bio.PDB.mmtf import MMTFParser

  
import matplotlib.pyplot as plt
import json
import pickle
import networkx as nx
from collections import OrderedDict
from lifelines.utils import concordance_index as ci

ModuleNotFoundError: ignored

## 2. Parsing Data
TODO: describe the data we are using.

### 2.1 Parsing PDB data


In [34]:


pdb_data = pd.read_csv('PDBbind2019PLdata.csv')
pdb_proteinName = np.array(pdb_data['proteinName'])
pdb_affinity = np.array(pdb_data['affinity'])
pdb_ligandName = np.array(pdb_data['ligandName'])



pdb_data_small = pd.read_csv('pdbbind_core_df_cleaned.csv')
pdb_proteinName_small = np.array(pdb_data_small['proteinName'])
pdb_affinity_small = np.array(pdb_data_small['affinity'])
pdb_ligandName_small = np.array(pdb_data_small['ligandName'])

In [35]:
print("pdb_proteinName.shape:", pdb_proteinName.shape)
print("pdb_affinity.shape:", pdb_affinity.shape)
print("pdb_ligandName.shape:", pdb_ligandName.shape)

print("pdb_proteinName_small.shape:", pdb_proteinName_small.shape)
print("pdb_affinity_small.shape:", pdb_affinity_small.shape)
print("pdb_ligandName_small.shape:", pdb_ligandName_small.shape)

pdb_proteinName.shape: (17679,)
pdb_affinity.shape: (17679,)
pdb_ligandName.shape: (17679,)
pdb_proteinName_small.shape: (193,)
pdb_affinity_small.shape: (193,)
pdb_ligandName_small.shape: (193,)


 #### 3D Representation 1: Contact map

We get the list of residues in each protein and treat that as a single chain to calculate the contact map of this protein. After flattened, the length of the contact map vector should be **O(residue_count * residue_count)**, which is too big for some proteins with large number of residues. Hence, we tried to shrink it by collapsing the 3D coordinates a group of neighboring residues into a single one.

In [36]:



def filter_residues(residues):
  new_residues = []
  for res in residues:
    if 'CA' in res:
      new_residues.append(res)
  return new_residues

def calc_residue_dist_by_residues(residue_one, residue_two) :
    """Returns the C-alpha distance between two residues"""
    diff_vector  = residue_one["CA"].coord - residue_two["CA"].coord
    return np.sqrt(np.sum(diff_vector * diff_vector))

#Essentially the Euclidean distance between 2 coords
def calc_residue_dist_by_coords(coord1, coord2) :
    """Returns the C-alpha distance between two residues"""
    diff_vector  = coord1 - coord2
    # return np.sqrt(np.sum(diff_vector * diff_vector))
    return np.linalg.norm(diff_vector)



# We expect residue_list1 and residue_list2 to be the same thing: residue list of one protein. Hence the result should be a symmetric matrix, and thus we decided to only calculate the independent entries.
def calc_dist_matrix_by_residues(residue_list1) :
    """Returns a matrix of C-alpha distances between two chains"""
    # answer = np.zeros((len(residue_list1), len(chain_two)), np.float)
    answer = []
    for row, residue_one in enumerate(residue_list1) :
        for col, residue_two in enumerate(residue_list1[row+1:]) :
            answer.append(calc_residue_dist(residue_one, residue_two))
    return answer

def calc_dist_matrix_by_coords(coord_list1) :
    """Returns a matrix of C-alpha distances between two chains"""
    # answer = np.zeros((len(residue_list1), len(chain_two)), np.float)
    answer = []
    for row, coord1 in enumerate(coord_list1) :
        for col, coord2 in enumerate(coord_list1[row+1:]) :
            answer.append(calc_residue_dist_by_coords(coord1, coord2))
    return answer


# Returns the contact map for one protein and prints the info - slow (for demonstration purpose)
# It does the shrinking by shrinking_ratio within each chain (coords from end of chain1 and start of chain 2 will NOT be collapsed together).
def get_protein_contact_map_slow(proteinName, shrinking_ratio=20):
  structure=MMTFParser.get_structure_from_url(proteinName)
  all_chains =structure[0]
  
  #Get 1.residues: list of all residues in the protein
#      2.CA_coord_list has length equal num of chains. 
  residues = []
  CA_coord_list = []

  for idx_chain, chain in enumerate(all_chains):
    CA_curr_chain = []
    for idx_residue, residue in enumerate(chain):
      residues.append(residue)
      if 'CA' in residue:
        CA_curr_chain.append(residue['CA'].coord)
    CA_coord_list.append(CA_curr_chain)
    print("Chain with id %d has %d residues in total, and %d residues with CA." % (idx_chain, len(chain), len(CA_curr_chain)))
  print("\nCount of residues: ", len(residues))
  residues = filter_residues(residues)
  print("Count of residues with CA: ", len(residues))


  # Get coord_list: collapsing list of CA coords of all resiudues with CA in the protein
  coord_list = []
  for CA_curr_chain in CA_coord_list:
    cutting_points = list(np.arange(0,len(CA_curr_chain),shrinking_ratio))
    cutting_points.append(len(CA_curr_chain))
    for i, start in enumerate(cutting_points[:-1]):
      end = cutting_points[i+1]
      coord_list.append(np.mean(CA_curr_chain[start:end],axis=0))
  print("After shrinking, len(coord_list) = ", len(coord_list))


  # Calculate contact_map: pair wise distance of the coords
  contact_map = []
  for coord1 in coord_list:
    for coord2 in coord_list:
      dist = calc_residue_dist_by_coord(coord1, coord2)
      contact_map.append(dist)
  print("len(contact_map) = ", len(contact_map), end="\n\n")
  return contact_map



# Returns the contact map for one protein and does not print the info - fast
# It does not do the exact shrinking within each chain (That is, coords from end of chain1 and start of chain 2 will be collapsed together).
def get_protein_contact_map(proteinName, shrinking_ratio=20):
  structure=MMTFParser.get_structure_from_url(proteinName)
  all_chains =structure[0]
  
  #Get CA_coord_list: list of CA coords of all residues with CA
  CA_coord_list = []
  for idx_chain, chain in enumerate(all_chains):
    for idx_residue, residue in enumerate(chain):
      if 'CA' in residue:
        CA_coord_list.append(residue['CA'].coord)
  # print("len(CA_coord_list) = ", len(CA_coord_list))


  # Get coord_list: collapsing list of CA coords of all resiudues with CA in the protein
  coord_list = []
  cutting_points = list(np.arange(0,len(CA_coord_list),shrinking_ratio))
  cutting_points.append(len(CA_curr_chain))
  for i, start in enumerate(cutting_points[:-1]):
    end = cutting_points[i+1]
    coord_list.append(np.mean(CA_coord_list[start:end],axis=0))
  # print("len(coord_list) = ", len(coord_list))

  # Calculate contact_map: pair wise distance of the coords
  contact_map = []
  contact_map = calc_dist_matrix_by_coords(coord_list)
  # print("len(contact_map) = ", len(contact_map))

  return contact_map


In [37]:
def foo(proteinName):
  all_chains=MMTFParser.get_structure_from_url(proteinName)[0]
  CA_coord_list = []
  for idx_chain, chain in enumerate(all_chains):
    for idx_residue, residue in enumerate(chain):
      if 'CA' in residue:
        CA_coord_list.append(residue['CA'].coord)

  return 

for i in range(len(pdb_proteinName)):
  foo(pdb_proteinName[i])
  
  if i %10==0:
    print(i, end=",")
  


NameError: ignored

In [ ]:
map = get_protein_contact_map_slow(pdb_proteinName[0],shrinking_ratio=SHRINKING_RATIO)


In [ ]:
# Get contact maps for all proteins:

SHRINKING_RATIO = 20 #after exploring, 20 is a good SHRINKING_RATIO to shrink the contact maps into reasonable lengths
contact_maps = []
lens = []
for i in range(len(pdb_proteinName)):
  map = get_protein_contact_map(pdb_proteinName_small[i],shrinking_ratio=SHRINKING_RATIO)
  contact_maps.append(map)
  lens.append(len(map))
  
  if i %10==0:
    print(i, end=",")
  
print(np.min(lens), np.max(lens))
  


In [ ]:
# Now do cutting/padding on the data so they have equal length:
#TODO: use the code from miniproject (how they got 1000-long vector for proteins)

Note: After optimizing the code as much as we can, getting the contact maps of proteins is still slow because of some unavoidable calculations. Each protein takes about **1 sec** to calculate its contact map, which representation takes **???#TODO sec** to calculate for all 17680 proteins.

Hence, we will only sample the core binding triples from 'pdbbind_core_df.csv' and we will compare performances of Representation 1 and Representation 2 on this dataset.

For the general dataset 'PDBbind2019PLdata.csv',Representation 2 is the only option we have.

 #### 3D Representation 2: Distances of residues from the center.
We assume a center for the protein, which is calculated by averaging the 3D coordinates of the residues. Then we use the distances of residues from this center as our 3D representation of this protein. One advantage of this representation is that the final length is of order **O(residue_count)**.


In [ ]:
def get_center_distances(proteinName, printInfo=False):
  all_chains=MMTFParser.get_structure_from_url(proteinName)[0]
  

  #Step1: Get CA_coord_list: list of CA coords of all residues with CA
  CA_coord_list = []
  for idx_chain, chain in enumerate(all_chains):
    for idx_residue, residue in enumerate(chain):
      if 'CA' in residue:
        CA_coord_list.append(residue['CA'].coord)
  if printInfo:
    print("len(CA_coord_list) = ", len(CA_coord_list))

  #Step2: average coords to get center
  center = np.mean(CA_coord_list, axis=0)
  if printInfo:
    print("center = ", center)


  #Step3: calculate distances from the center
  distances = []
  for coord in CA_coord_list:
    dist = calc_residue_dist_by_coords(center, coord)#Essentially the Euclidean distance between 2 coords
    distances.append(dist)
  if printInfo:
    print("len(distances) = ", len(distances))

  return distances

 

In [ ]:
d0 = get_center_distances(pdb_proteinName[0], printInfo=True)
print()
d1 = get_center_distances(pdb_proteinName[1], printInfo=True)

In [ ]:
# Get contact maps for all proteins:

SHRINKING_RATIO = 20 #after exploring, 20 is a good SHRINKING_RATIO to shrink the contact maps into reasonable lengths
distances = []
lens = []
for i in range(len(pdb_proteinName)):
  d = get_center_distances(pdb_proteinName[i])
  distances.append(d)
  lens.append(len(d))
  
  if i %10==0:
    print(i, end=",")
  


In [ ]:
print(np.min(lens), np.max(lens))

In [ ]:
# Now do cutting/padding on the data so they have equal length:
#TODO: use the code from miniproject (how they got 1000-long vector for proteins)

### 2.2 Parsing KIBA data

In [ ]:
# for converting protein sequence to categorical format
seq_voc = "ABCDEFGHIKLMNOPQRSTUVWXYZ"
seq_dict = {v:i for i,v in enumerate(seq_voc)}
seq_dict_len = len(seq_dict)
max_seq_len = 1000   # Note that all protein data will have the same length 1000 

def seq_to_cat(prot):
    x = np.zeros(max_seq_len)
    for i, ch in enumerate(prot[:max_seq_len]): 
        x[i] = seq_dict[ch]
    return x  

# Concordance index evaluation. XM: given function is too slow, we use concordance_index from lifelines.utils to calculate CI score


We read in the ligands and the proteins, as well as the binding data.

In [ ]:
fpath = 'data_Drug_target_binding_affinity/data/kiba/'

# Read in drugs and proteins
drugs_ = json.load(open(fpath + "ligands_can.txt"), object_pairs_hook=OrderedDict)
drugs = np.array([Chem.MolToSmiles(Chem.MolFromSmiles(d),isomericSmiles=True) for d in drugs_.values()])
proteins_ = json.load(open(fpath + "proteins.txt"), object_pairs_hook=OrderedDict)
proteins = np.array(list(proteins_.values()))

# Read in affinity data
affinity = np.array(pickle.load(open(fpath + "Y","rb"), encoding='latin1'))

# Read in train/test fold  
train_fold = json.load(open(fpath + "folds/train_fold_setting1.txt"))
train_fold = [ee for e in train_fold for ee in e ]    
'''
Here all validation folds are aggregated into training set. 
If you want to train models with different architectures and/or 
optimize for model hyperparameters, we encourage you to use 5-fold 
cross validation as provided here.
'''
test_fold = json.load(open(fpath + "folds/test_fold_setting1.txt"))

# Prepare train/test data with fold indices
rows, cols = np.where(np.isnan(affinity)==False) 
drugs_tr = drugs[rows[train_fold]]
proteins_tr = np.array([seq_to_cat(p) for p in proteins[cols[train_fold]]])
affinity_tr = affinity[rows[train_fold], cols[train_fold]]

drugs_ts = drugs[rows[test_fold]]
proteins_ts = np.array([seq_to_cat(p) for p in proteins[cols[test_fold]]])
affinity_ts = affinity[rows[test_fold], cols[test_fold]]    

In [ ]:
NBITS = 1000

In [ ]:
print('XM: Now each protein is represented as a 1000 float array, but each drug is still SMILES string, we convert it to ECFP vectors')

# Convert to ECFP fingerprint
drugs_tr_ECFP = []
for i,drug in enumerate(drugs_tr):
  molecule = MolFromSmiles(drug)
  ECFP2 = Chem.GetMorganFingerprintAsBitVect(molecule, 2, nBits=NBITS).ToBitString()
  ECFP2arr = np.array(list(map(int, ECFP2)))
  drugs_tr_ECFP.append(ECFP2arr)
  if i % 1000==0:
    print(i, end=",")

drugs_tr_ECFP = np.array(drugs_tr_ECFP)
print(drugs_tr_ECFP.shape)

In [ ]:
print(drugs_tr_ECFP.shape)
print(proteins_tr.shape)
print(affinity_tr.shape)

In [ ]:
drugs_ts_ECFP = []
for i,drug in enumerate(drugs_ts):
  molecule = MolFromSmiles(drug)
  ECFP2 = Chem.GetMorganFingerprintAsBitVect(molecule, 2, nBits=NBITS).ToBitString()
  ECFP2arr = np.array(list(map(int, ECFP2)))
  drugs_ts_ECFP.append(ECFP2arr)
  if i % 1000==0:
    print(i, end=",")

drugs_ts_ECFP = np.array(drugs_ts_ECFP)
print(drugs_ts_ECFP.shape)


print(drugs_tr_ECFP.shape)
print(proteins_ts.shape)
print(affinity_ts.shape)


In [ ]:
pair_tr=[]
N=len(drugs_tr_ECFP)
print("N = ", N)

for i in range(N):
  new_pair_tr=np.concatenate((proteins_tr[i], drugs_tr_ECFP[i]))
  # new_pair_tr=np.array([proteins_tr[i], drugs_tr_ECFP[i]])
  pair_tr.append(new_pair_tr)
pair_tr = np.array(pair_tr)
print("pair_tr(concatenated).shape = ", pair_tr.shape)
print("affinity_tr.shape = ", affinity_tr.shape)

In [ ]:
pair_ts=[]
N=len(drugs_ts_ECFP)
print("N = ", N)

for i in range(N):
  new_pair_ts=np.concatenate((proteins_ts[i], drugs_ts_ECFP[i]))
  pair_ts.append(new_pair_ts)
pair_ts = np.array(pair_ts)
print("pair_ts(concatenated).shape = ", pair_ts.shape)
print("affinity_ts.shape = ", affinity_ts.shape)

## 3. Train a model on KIBA data

In [ ]:

# First we define a keras sequential model
keras_model = tf.keras.Sequential([
    tf.keras.layers.InputLayer(input_shape=(1000+NBITS,)),
    # tf.keras.layers.Flatten(input_shape=(2,)), (XM: actually our concatenation has done the flatten step) 
    tf.keras.layers.Dense(1024, activation='relu'),
    # tf.keras.layers.Dense(16, activation='relu'),
    # tf.keras.layers.Dropout(rate=0.2),
    tf.keras.layers.Dense(1)
])

keras_model2 = tf.keras.Sequential([
    tf.keras.layers.InputLayer(input_shape=(1000+NBITS,)),
    # tf.keras.layers.Flatten(input_shape=(2,)), (XM: actually our concatenation has done the flatten step) 
    tf.keras.layers.Dense(1024, activation='relu'),
    tf.keras.layers.Dense(512, activation='relu'),
    tf.keras.layers.Dense(256, activation='relu'),
    tf.keras.layers.Dense(128, activation='relu'),
    tf.keras.layers.Dense(64, activation='relu'),
    tf.keras.layers.Dense(16, activation='relu'),
    tf.keras.layers.Dense(4, activation='relu'),
    tf.keras.layers.Dropout(rate=0.2),
    tf.keras.layers.Dense(1)
])

# loss_fn = tf.keras.losses.MSE()

In [ ]:
def train_test_model(model, pair_tr, affinity_tr, pair_ts, affinity_ts, epochs=20, calculateCI=True):
  model.compile(optimizer='adam',     # This object specifies the training procedure
              loss= 'mean_squared_error',         # The function to minimize during optimization
              metrics=['mse']) # Used to monitor training. XM: metrics=[ci] gives bug (may due to the numpy calculation inside of ci(y,f), Google says we might need to downgrade numpy to lower than 1.20: pip install numpy==1.19. However, let's just calculate CI by ourselves in the end

  model.fit(pair_tr, affinity_tr, epochs=epochs)

  print("Evaluation:")
  print('training MSE:', model.evaluate(pair_tr, affinity_tr))
  print('test MSE:', model.evaluate(pair_ts, affinity_ts))
  if calculateCI:
    print('test CI:', ci(affinity_ts, model.predict(pair_ts)))


In [ ]:
train_test_model(keras_model, pair_tr, affinity_tr, pair_ts, affinity_ts, epochs=20, calculateCI=True)

In [ ]:
train_test_model(keras_model2, pair_tr, affinity_tr, pair_ts, affinity_ts, epochs=20, calculateCI=True)

####  Try using deepchem keras:

In [ ]:

def train_test_model_dc(model_keras, pair_tr, affinity_tr, pair_ts, affinity_ts, epochs=20, calculateCI=True):
  # train_dataset = dc.data.NumpyDataset(X=pair_tr, y=affinity_tr.reshape(len(affinity_tr), 1))
  # test_dataset = dc.data.NumpyDataset(X=pair_ts, y=affinity_ts.reshape(len(affinity_ts), 1))
  # dc_model = dc.models.KerasModel(model_keras, dc.models.losses.L2Loss()) # we pass the model and a dc loss funciton
  
  # dc_model.fit(train_dataset, nb_epoch=epochs)
  # metric = dc.metrics.Metric(dc.metrics.mean_squared_error)
  # print("Evaluation:")
  # print('training MSE:', dc_model.evaluate(train_dataset, [metric]))
  # print('test MSE:',dc_model.evaluate(test_dataset, [metric]))
  # if calculateCI:
  #   print('test CI:', ci(affinity_ts, dc_model.predict(test_dataset)))
  
  #TODO: Install deepchem takes too long every time. Here are temporarily usee keras NN for training. 
  # After finishing all coding. Comment out the code below and use the code above!

  return train_test_model(model_keras, pair_tr, affinity_tr, pair_ts, affinity_ts, epochs=epochs, calculateCI=calculateCI)


In [ ]:
train_test_model_dc(keras_model2, pair_tr, affinity_tr, pair_ts, affinity_ts, epochs=20, calculateCI=True)

[The original paper](https://www.ncbi.nlm.nih.gov/pmc/articles/PMC4364066/) obtained CI score of 0.782 and MSE of 0.411, and [random forest model](https://www.ncbi.nlm.nih.gov/pmc/articles/PMC5395521/) got 0.836 and 0.222. Can you beat them?

Miniproject notes：
* Protein 1000; Drug 2048; tf Keras:

 **Training: MSE=0.704 (epoch=10); Test: MSE=0.679, CI=0.5**

* Protein 1000; Drug 1000; tf Keras:
   
   **Training: MSE=0.390 (epoch=20); Test: MSE=0.391, CI=0.765**

* Try more complicated NN (keras_model2):

  **Training: MSE=0.356 (epoch=20); Test: MSE=0.359, CI=0.773**

* Protein 1000; Drug 1000; dc model with (keras_model2):
   
   **Training: MSE=0.303 (epoch=20); Test: MSE=0.316, CI=0.782**

* TODO: COVID-19 prediction (HIV score seems not very good??)
* TODO: Julia + Decision Tree


## 4. Use  your model on COVID-19 protease

Now, use your trained model to identify drugs that could be used as COVID-19 protease inhibitors from these 2111 drugs in the dataset. The sequence of the protease is provided in `6Y84_A.fasta.txt`. You might want to first predict a binding affinity of Ritonavior, a well known HIV drug that binds to HIV protease, to get the sense of a good binding score for this task. SMILES of Ritonavior is provided below.  

In [ ]:
def protein2vec(protein_seq):
  return seq_to_cat(protein_seq)

def drug2vec(drug_smiles, nBits=NBITS):
  molecule = MolFromSmiles(drug_smiles)
  ECFP2 = Chem.GetMorganFingerprintAsBitVect(molecule, 2, nBits).ToBitString()#XM: default nBits=2048
  drug_ECFP = np.array(list(map(int, ECFP2)))
  return drug_ECFP


# train_dataset = dc.data.NumpyDataset(X=pair_tr, y=affinity_tr.reshape(len(affinity_tr), 1))
# dc_model = dc.models.KerasModel(keras_model, dc.models.losses.L2Loss()) # we pass the model and a dc loss funciton
# dc_model.fit(train_dataset, nb_epoch=20)
# print("dc_model is ready!")

# def protein_drug_predict_dc(protein_vec, drug_vec):#XM: this doesn't work well on HIV
#   protein_drug_pair = np.concatenate((protein_vec, drug_vec))
#   protein_drug_pair = protein_drug_pair.reshape((1, len(protein_drug_pair)))

#   test_dataset = dc.data.NumpyDataset(X=protein_drug_pair)
#   return dc_model.predict(test_dataset)[0][0]

keras_model.compile(optimizer='adam',  
              loss= 'mean_squared_error', 
              metrics=['mse']) 
keras_model.fit(pair_tr, affinity_tr, epochs=20)
print("keras_model is ready!")

keras_model2.compile(optimizer='adam',  
              loss= 'mean_squared_error', 
              metrics=['mse']) 
keras_model2.fit(pair_tr, affinity_tr, epochs=20)
print("keras_model2 is ready!")

def protein_drug_predict(protein_vec, drug_vec, model):
  protein_drug_pair = np.concatenate((protein_vec, drug_vec))
  protein_drug_pair = protein_drug_pair.reshape((1, len(protein_drug_pair)))
  
  return model.predict([protein_drug_pair])[0][0]


In [ ]:
# HIV: https://www.uniprot.org/uniprot/O90777; https://www.uniprot.org/uniprot/O90777/protvista
HIV_prot = 'PQVTLWQRPIVTIKIGGQLKEALLDTGADDTVLEEMSLPGKWKPKMIGGIGGFIKVRQYDQVSIEICGHKAIGTVLIGPTPVNIIGRNLLTQLGCTLNF'
HIV_vec = seq_to_cat(HIV_prot)
print(HIV_vec.shape)

ritonavior = 'CC(C)C1=NC(=CS1)CN(C)C(=O)NC(C(C)C)C(=O)NC(CC2=CC=CC=C2)CC(C(CC3=CC=CC=C3)NC(=O)OCC4=CN=CS4)O'
ritonavior_ECFP = drug2vec(ritonavior)
print(ritonavior_ECFP.shape)

In [ ]:
print("HIV-Ritonavior binding score:")
HIV_Ritonavior_score = protein_drug_predict(HIV_vec, ritonavior_ECFP, keras_model2)
HIV_Ritonavior_score

In [ ]:
print("HIV-AnotherDrug binding scores:")

HIV_affinities = []
for i,drug in enumerate(drugs):
  drug_ECFP = drug2vec(drug)
  aff = protein_drug_predict(HIV_vec, drug_ECFP, keras_model2)
  HIV_affinities.append(aff)

HIV_affinities = np.array(HIV_affinities)
print(HIV_affinities[:20])
print(sorted(np.array(list(set(HIV_affinities)))))

print("\n\nHow many drugs have better bindings with HIV, compared to Ritonavior?")
print(len(HIV_affinities[HIV_affinities > HIV_Ritonavior_score]) / len(HIV_affinities))
print("How many drugs have equal bindings with HIV, compared to Ritonavior?")
print(len(HIV_affinities[HIV_affinities == HIV_Ritonavior_score]) / len(HIV_affinities))

* When using 1000-vector for drugs:

   It seems almost 70% out of the 2111 drugs bind better with HIV, compared to Ritonavior. Thus our model might not be good enough?

* When using 1000-vector for drugs:

  Indeed Ritonavior has the highest binding score, but 94% of the 2111 drugs have the same binding score...


In [ ]:
COVID_list = []
with open('data_Drug_target_binding_affinity/data/6Y84_A.fasta.txt', 'r') as f:
    for line in f:
        COVID_list.append(line[:-1])

COVID_seq = ''
for s in COVID_list[1:]:
  COVID_seq += s
print("COVID-19 amino acids sequence: ", COVID_seq)
COVID_vec = seq_to_cat(COVID_seq)

In [ ]:
print("COVID-Drug binding scores:")
COVID_affinities = []
for i,drug in enumerate(drugs):
  drug_ECFP = drug2vec(drug)
  aff = protein_drug_predict(COVID_vec, drug_ECFP, keras_model2)
  COVID_affinities.append(aff)

COVID_affinities = np.array(COVID_affinities)
print(COVID_affinities[:20])

In [ ]:
print("Below are the Top 5 drug candidates for COVID-19:")
print("Affinitiy score       Drug smiles: ")
for idx in np.argsort(COVID_affinities)[-5:]:
  drug_smiles = drugs[idx]
  aff_score = COVID_affinities[idx]
  print(aff_score, "         ",  drug_smiles)

## 5. Dimension reduction.

Now because the number of data samples is large. It takes quite long to run NN without GPU. Hence, let's investigate how well the reduced data would perform in our NN model. 

#### 5.1. Reduction by Autoencoder
###### 5.1.1 Try use 2 Autoencoders for pair_tr and pair_ts


#### 5.2. Reduction by PCA


In [ ]:
from sklearn.decomposition import PCA

pca = PCA(n_components=128)
pair_tr_pca = pca.fit_transform(pair_tr)
print("pair_tr_pca shape:", pair_tr_pca.shape)
print("Explained how much variance:", pca.explained_variance_ratio_.cumsum()[-1])

pca = PCA(n_components=128)
pair_ts_pca = pca.fit_transform(pair_ts)
print("\npair_ts_pca shape:", pair_ts_pca.shape)
print("Explained how much variance:", pca.explained_variance_ratio_.cumsum()[-1])

In [ ]:
train_test_model(keras_model2, pair_tr_pca, affinity_tr, pair_ts_pca, affinity_ts, epochs=20, calculateCI=True)

We see after reduction by Autoencoder or PCA, our NN still gives relatively good results.